In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Loading my API Credentials
with open('/Users/chery/.secret/yelp_api.json') as file:
    login = json.load(file)
# Instantiating the YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# defining parameters for the API call 
LOCATION = 'Los Angeles'
TERM = 'Mexican'

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_LosAngeles_Mexican.json"
JSON_FILE

'Data/results_in_progress_LosAngeles_Mexican.json'

In [5]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
                
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [6]:
# calling create/delete file function
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,"r") as f:
    previous_results = json.load(f)
    
## setting the offset based on previous saved results
n_results = len(previous_results)
print(f"- {n_results} previous results found.")
# using the yelp_api intance search_query method to perform an API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## What is our total for results?
total_results = results["total"]
## How many details have we retrieved so far?
results_per_page = len(results["businesses"])
# Using math.ceil to round up to get the totalnumber of pages.
n_pages = math.ceil((results["total"]-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_LosAngeles_Mexican.json not found. Saving empty list to new file.
- 0 previous results found.


510

In [7]:
# 
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, "r") as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 500:
        print("Exceeded 500 API calls. Stopping loop.")
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results["businesses"])
    
    # display(previous_results)
    with open(JSON_FILE,"w") as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/510 [00:00<?, ?it/s]

Exceeded 500 API calls. Stopping loop.


In [8]:
# converting and loading json as a dataframe
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,IzRINfSLBr6k_dq1e2WNew,guelaguetza-restaurante-los-angeles,Guelaguetza Restaurante,https://s3-media1.fl.yelpcdn.com/bphoto/36frnk...,False,https://www.yelp.com/biz/guelaguetza-restauran...,2847,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 34.05241, 'longitude': -118.30071}","[pickup, delivery]",$$,"{'address1': '3014 W Olympic Blvd', 'address2'...",+12134270608,(213) 427-0608,2164.088317
1,LNb-ElJwKIXS6Lx5xdRB1Q,toca-madera-los-angeles-los-angeles,Toca Madera - Los Angeles,https://s3-media3.fl.yelpcdn.com/bphoto/ccCxvb...,False,https://www.yelp.com/biz/toca-madera-los-angel...,3140,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.0,"{'latitude': 34.072969, 'longitude': -118.374972}",[],$$$,"{'address1': '8450 W 3rd St', 'address2': '', ...",+13238529400,(323) 852-9400,5093.966283
2,2t-FTIdnJC1gDjpims7oMQ,el-huero-los-angeles-2,El Huero,https://s3-media1.fl.yelpcdn.com/bphoto/Mkvtu2...,False,https://www.yelp.com/biz/el-huero-los-angeles-...,1227,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 34.02479918, 'longitude': -118.27...","[pickup, delivery]",$$,"{'address1': '3000 S Figueroa St', 'address2':...",+12137470012,(213) 747-0012,5728.629465
3,Lop79P2KM9zFUCCaBYz6zA,tlayuda-l-a-restaurant-los-angeles-6,Tlayuda L.A. Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/g0x_Xh...,False,https://www.yelp.com/biz/tlayuda-l-a-restauran...,867,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 34.0906351, 'longitude': -118.307...","[pickup, delivery]",$$,"{'address1': '5450 Santa Monica Blvd', 'addres...",+13239635045,(323) 963-5045,3458.207351
4,WJzD72vgqjH7BHE0UF6N4g,tu-madre-los-feliz-los-angeles-3,Tu Madre - Los Feliz,https://s3-media4.fl.yelpcdn.com/bphoto/DFX8l_...,False,https://www.yelp.com/biz/tu-madre-los-feliz-lo...,120,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",4.5,"{'latitude': 34.10453972993196, 'longitude': -...","[pickup, delivery]",NaN,"{'address1': '1824 Hillhurst Ave', 'address2':...",+13234269031,(323) 426-9031,5724.372459


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
495,FvLCPQ6Jsn94vUhKE9_FXg,mi-ranchito-veracruz-cocktails-los-angeles-3,Mi Ranchito Veracruz Cocktails,https://s3-media2.fl.yelpcdn.com/bphoto/8LNmzh...,False,https://www.yelp.com/biz/mi-ranchito-veracruz-...,3,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.5,"{'latitude': 34.10184162189683, 'longitude': -...","[pickup, delivery]",NaN,"{'address1': '6679 Hollywood Blvd', 'address2'...",+13233662760,(323) 366-2760,4667.802361
496,ePBR5sUMgUg9hFW20d5jeA,manzke-los-angeles,Manzke,https://s3-media4.fl.yelpcdn.com/bphoto/KAuUdC...,False,https://www.yelp.com/biz/manzke-los-angeles?ad...,88,"[{'alias': 'modern_european', 'title': 'Modern...",4.5,"{'latitude': 34.055389703322575, 'longitude': ...",[],$$$$,"{'address1': '9575 W Pico Blvd', 'address2': '...",,,7055.296105
497,iojZkvl5g6tFHEm1XkN1OA,tacos-toluca-los-angeles-5,Tacos Toluca,https://s3-media2.fl.yelpcdn.com/bphoto/yLqehq...,False,https://www.yelp.com/biz/tacos-toluca-los-ange...,3,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",5.0,"{'latitude': 34.03815, 'longitude': -118.26028}",[],NaN,"{'address1': 'W 12th St & S Broadway', 'addres...",,,6202.801659
498,l54HPAd5PraebhSmaB36WA,tacos-los-carnales-los-angeles,Tacos Los Carnales,https://s3-media1.fl.yelpcdn.com/bphoto/S_gbD4...,False,https://www.yelp.com/biz/tacos-los-carnales-lo...,69,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 34.0041, 'longitude': -118.256164}",[delivery],$,"{'address1': '4370 S Central Ave', 'address2':...",+13238461135,(323) 846-1135,8765.251303
499,DTXC9PtF1k6vcJJyUoKESw,house-of-birria-culver-city,House Of Birria,https://s3-media3.fl.yelpcdn.com/bphoto/-bsHm8...,False,https://www.yelp.com/biz/house-of-birria-culve...,11,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",4.5,"{'latitude': 33.98720579312247, 'longitude': -...","[pickup, delivery]",NaN,"{'address1': '5660 Selmaraine Dr', 'address2':...",+19512644266,(951) 264-4266,10787.409187


In [9]:
# checking for duplicates using subset id 
final_df.duplicated(subset="id").sum()

3

In [10]:
# dropping duplicates
final_df = final_df.drop_duplicates(subset="id")
final_df.duplicated(subset="id").sum()

0

In [11]:
# saving final file as a csv
final_df.to_csv("Data/final_results_LosAngeles_Mexican.csv.gz", compression="gzip",index=False)

In [12]:
# displaying final df
dispdf = pd.read_csv("Data/final_results_LosAngeles_Mexican.csv.gz")
dispdf.head(20)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,IzRINfSLBr6k_dq1e2WNew,guelaguetza-restaurante-los-angeles,Guelaguetza Restaurante,https://s3-media1.fl.yelpcdn.com/bphoto/36frnk...,False,https://www.yelp.com/biz/guelaguetza-restauran...,2847,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 34.05241, 'longitude': -118.30071}","['pickup', 'delivery']",$$,"{'address1': '3014 W Olympic Blvd', 'address2'...",1.213427e+10,(213) 427-0608,2164.088317
1,LNb-ElJwKIXS6Lx5xdRB1Q,toca-madera-los-angeles-los-angeles,Toca Madera - Los Angeles,https://s3-media3.fl.yelpcdn.com/bphoto/ccCxvb...,False,https://www.yelp.com/biz/toca-madera-los-angel...,3140,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.0,"{'latitude': 34.072969, 'longitude': -118.374972}",[],$$$,"{'address1': '8450 W 3rd St', 'address2': '', ...",1.323853e+10,(323) 852-9400,5093.966283
2,2t-FTIdnJC1gDjpims7oMQ,el-huero-los-angeles-2,El Huero,https://s3-media1.fl.yelpcdn.com/bphoto/Mkvtu2...,False,https://www.yelp.com/biz/el-huero-los-angeles-...,1227,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 34.02479918, 'longitude': -118.27...","['pickup', 'delivery']",$$,"{'address1': '3000 S Figueroa St', 'address2':...",1.213747e+10,(213) 747-0012,5728.629465
3,Lop79P2KM9zFUCCaBYz6zA,tlayuda-l-a-restaurant-los-angeles-6,Tlayuda L.A. Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/g0x_Xh...,False,https://www.yelp.com/biz/tlayuda-l-a-restauran...,867,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 34.0906351, 'longitude': -118.307...","['pickup', 'delivery']",$$,"{'address1': '5450 Santa Monica Blvd', 'addres...",1.323964e+10,(323) 963-5045,3458.207351
4,WJzD72vgqjH7BHE0UF6N4g,tu-madre-los-feliz-los-angeles-3,Tu Madre - Los Feliz,https://s3-media4.fl.yelpcdn.com/bphoto/DFX8l_...,False,https://www.yelp.com/biz/tu-madre-los-feliz-lo...,120,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",4.5,"{'latitude': 34.10453972993196, 'longitude': -...","['pickup', 'delivery']",NaN,"{'address1': '1824 Hillhurst Ave', 'address2':...",1.323427e+10,(323) 426-9031,5724.372459
5,Pb9KCJOvzfE20vRmOGtOrg,el-coyote-cafe-los-angeles,El Coyote Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/GqKKe_...,False,https://www.yelp.com/biz/el-coyote-cafe-los-an...,2164,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,"{'latitude': 34.0758743286133, 'longitude': -1...",['delivery'],$$,"{'address1': '7312 Beverly Blvd', 'address2': ...",1.323939e+10,(323) 939-2255,3019.394471
6,6Dd4kTDiikEB3wQp___0fA,taqueria-los-anaya-los-angeles,Taqueria Los Anaya,https://s3-media2.fl.yelpcdn.com/bphoto/4zjN9D...,False,https://www.yelp.com/biz/taqueria-los-anaya-lo...,1478,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 34.0327911, 'longitude': -118.341...",[],$$,"{'address1': '4651 W Adams Blvd', 'address2': ...",1.323731e+10,(323) 731-4070,3719.047564
7,kr9RZATtNwJznY9TaMexUA,tekila-los-angeles,Te'Kila,https://s3-media3.fl.yelpcdn.com/bphoto/qOUu4v...,False,https://www.yelp.com/biz/tekila-los-angeles?ad...,1211,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 34.101795, 'longitude': -118.3323...","['pickup', 'delivery']",$$,"{'address1': '6541 Hollywood Blvd', 'address2'...",1.323960e+10,(323) 960-2404,4584.229210
8,bKeJWK8yxAo0x5y_0kGyVA,mercado-hollywood-los-angeles,Mercado Hollywood,https://s3-media3.fl.yelpcdn.com/bphoto/Pe3Ydb...,False,https://www.yelp.com/biz/mercado-hollywood-los...,841,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 34.13166, 'longitude': -118.35262}","['pickup', 'delivery']",$$$,"{'address1': '3413 Cahuenga Blvd W', 'address2...",1.323512e+10,(323) 512-2500,8303.472510
9,rX4XXEwzSea-EJxuMYAFvw,el-carmen-los-angeles,El Carmen,https://s3-media4.fl.yelpcdn.com/bphoto/O8tjt3...,False,https://www.yelp.com/biz/el-carmen-los-angeles...,1059,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude':